In [4]:
from glob import glob
import pyodbc
import re
import pandas as pd
#database
server = 'TOM'
database = 'CCH_HSC_08.22.23'
username = 'TOM'
password = 'Silversea2022@'
connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}"
connection = pyodbc.connect(connection_string)
query = " select ClientIdSubId , ClientId AS [Client #] , ClientGroupName [Client Group], ClientSortName [Client Name], CONCAT_WS(' ', AddressLine1, AddressLine2, AddressLine3) Address ,CityName [City] ,StateProvinceCode [State] ,PostalCode [ZipCode] ,IIF(LEN(C.FiscalPeriod) = 0 or C.FiscalPeriod is null, null, LEFT(C.FiscalPeriod,2)) [Year End] ,[Eng Ltr Fees if Included] AS [Eng Ltr Fees if Inlcluded] ,[Tax Engagement Letter Type] AS [Tax Eng Ltr Type] ,[States/Localities] AS [States/Localities] ,[Changes in States/Localities] AS [Changes in States/Localities] ,[AA Engagement Letter Type] AS [AA Eng Ltr Type] ,[AA Financial Framework (FF)] AS [AA Financial Framework (FF)] ,[AA Departure from FF?] AS [AA Departure from FF?] ,[AA Not Independent?] AS [AA Not Independent?] ,[Eng Ltr Contact] AS [Eng Ltr Contact] ,[Eng Ltr Contact Email] AS [Eng Ltr Contact Email] ,[OS Client] AS [OS Client] ,[CAL Contact] AS [CAL Contact] ,[CAL Contact Email] AS [CAL Contact Email] ,[CAL Contact 2] AS [CAL Contact 2] ,[CAL Contact Email 2] AS [CAL Contact Email 2] ,[Depreciation Schedule Sent w/ CAL?] AS [Depreciation Schedule Sent w/ CAL?] ,[Notes/Comments] AS [Other Comments, Changes, etc.] ,StaffName AS [Primary] from CLIENT C LEFT JOIN ( 	select ClientIdent, ClientGroupName from CLIENTGROUP 	where FinancialReportingGroupFlag = 't' ) CG ON CG.ClientIdent = C.ClientIdent LEFT JOIN ( 	select ReferenceIdent 	, IIF(AddressLine1='', null, AddressLine1) AddressLine1 	, IIF(AddressLine2='', null, AddressLine2) AddressLine2 	, IIF(AddressLine3='', null, AddressLine3) AddressLine3 	, IIF(CityName='', null, CityName) CityName 	, IIF(StateProvinceCode='', null, StateProvinceCode) StateProvinceCode 	, IIF(PostalCode='', null, PostalCode) PostalCode 	from CLIENTADDRESS 	WHERE PrimaryAddressFlag = 't' 	and ReferenceIdentType = 'ClientIntID' ) CA ON CA.ReferenceIdent  = C.ClientIdent LEFT JOIN ( 	select * from CLIENTPHONE 	WHERE PrimaryPhoneFlag = 't' 	and ReferenceIdentType = 'ClientIntID' 	and PhoneNumber <> '' ) CP ON CP.ReferenceIdent  = C.ClientIdent LEFT JOIN ( 	select * from CLIENTEMAIL 	WHERE PrimaryEmailAddressFlag = 't' 	and ReferenceIdentType = 'ClientIntID' 	and EmailAddress <> '' ) CE ON CE.ReferenceIdent  = C.ClientIdent LEFT JOIN ( select * FROM ( 	select ClientIdent, IIF(ClientCustomFieldValueDescription='',null,ClientCustomFieldValueDescription) ClientCustomFieldValueDescription,FirmCustomFieldLabel FROM CLIENTCUSTOMFIELD 	) T1 	PIVOT 	( 	MAX(ClientCustomFieldValueDescription) FOR FirmCustomFieldLabel 	IN ( [SIC], [Industry Team], [test], [Eng Ltr Contact Email], [County], [CAL Contact 2], [OS Client], [Eng Ltr Fees if Included], [CAL Contact], [CAL Contact Email 2], [Referred by], [Prior_NAICS], [AA Not Independent?], [Top Ten], [AA Financial Framework (FF)], [CAL Contact Email], [Tax Engagement Letter Type], [AA Engagement Letter Type], [Notes/Comments], [Annual Revenue], [Eng Ltr Contact], [AA Departure from FF?], [States/Localities], [OS Engagement Type], [# of Employees Here], [Industry SIC Code], [Changes in States/Localities], [Depreciation Schedule Sent w/ CAL?], [Number of Employees], [Prior_Entity], [Plan Name (EB Audit Eng Only)], [Custodian (EB Audit Eng Only)], [Referral Source], [Trade Name], [Client Sales]) ) PV1 ) CC ON CC.ClientIdent = C.ClientIdent LEFT JOIN ( SELECT ClientIdent, StaffName FROM CLIENTCRS LEFT JOIN STAFF S ON S.StaffIdent = CLIENTCRS.StaffIdent WHERE FirmClientStaffAssignmentName = 'Primary' ) CRS ON CRS.ClientIdent = C.ClientIdent "
sql = pd.read_sql_query(query, connection)
def convert_float_to_int_string(value):
    if isinstance(value, float):
        int_value = int(value)
        if value == int_value:
            return str(int_value)
    return str(value)


#data from excel
paths = sorted(glob(r'./New Data/*.xlsm'))
li = []
for path in paths:
    df = pd.read_excel(path)
    df['Source'] = path.split('\\')[1]
    li.append(df)
    df = pd.concat(li, ignore_index=True, axis=0)
df['Client #'] = df['Client #'].astype(str)
sql['Client #'] = sql['Client #'].astype(str)
sql['Client #'] = sql['Client #'].astype(str)
sql['Depreciation Schedule Sent w/ CAL?'] = sql['Depreciation Schedule Sent w/ CAL?'].replace('None','',inplace=True)
connection.close()
#replace none or nan ''
for column in df.columns:
    df[column].fillna('', inplace=True)
for column in sql.columns:
    sql[column].fillna('', inplace=True)

sql['Year End'] = sql['Year End'].apply(convert_float_to_int_string)
df['Year End'] = df['Year End'].apply(convert_float_to_int_string)
sql = sql.astype(str)
df = df.astype(str)
new_column_names = ['SQL_' + col for col in sql.columns]
sql.rename(columns=dict(zip(sql.columns, new_column_names)), inplace=True)

list_column_mapping_excel = ['Client Group', 'Client Name', 'Address', 'City', 'State',
       'ZipCode', 'Year End', 'Eng Ltr Fees if Inlcluded', 'Tax Eng Ltr Type',
       'States/Localities', 'Changes in States/Localities', 'AA Eng Ltr Type',
       'AA Financial Framework (FF)', 'AA Departure from FF?',
       'AA Not Independent?', 'Eng Ltr Contact', 'Eng Ltr Contact Email',
       'OS Client', 'CAL Contact', 'CAL Contact Email', 'CAL Contact 2',
       'CAL Contact Email 2', 'Depreciation Schedule Sent w/ CAL?',
       'Other Comments, Changes, etc.', 'Primary']
result = pd.merge(df, sql, how='left', left_on='Client #', right_on='SQL_Client #')
result = result.reset_index()



D= {}
for i in result.index:
    D[result.at[i,'index']] = []
    for j in list_column_mapping_excel:
        col_sql = 'SQL_' + j
        value_j = re.sub(r'[^a-zA-Z0-9]', '', str(result.at[i, j]).strip().lower())
        value_col_sql = re.sub(r'[^a-zA-Z0-9]', '', str(result.at[i, col_sql]).strip().lower())
        if value_j == value_col_sql:
#         if str(result.at[i,j]).strip().lower() == str(result.at[i,col_sql]).strip().lower():
            D[result.at[i,'index']].append('')
        else:
            D[result.at[i,'index']].append(j)
mapping = pd.DataFrame.from_dict(D, orient='index')
mapping.reset_index(inplace=True)
old_cols_name = list(range(25))
new_cols_name = ["Status " + column for column in list_column_mapping_excel]
mapping.rename(columns=dict(zip(old_cols_name, new_cols_name)), inplace=True)
mapping
result_last = pd.merge(result, mapping, how='left', left_on='index', right_on='index')
list_column = ['Source','Client #', 'SQL_ClientIdSubId']
for k in list_column_mapping_excel:
    list_column.append(k)
    list_column.append('SQL_' + k)
for h in list_column_mapping_excel:
    list_column.append('Status ' + h)
result_last = result_last[list_column]
result_last.to_csv('result_last.csv')

C:\Users\tomda\AppData\Local\Temp\ipykernel_30512\3526241673.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql = pd.read_sql_query(query, connection)
